## Fix colabs

The Colab notebooks are actually written in Jupyter or PyCharm.
This hides the code and clear the output.

In [ ]:
import json, os

folder = '.'
for filename in os.listdir(folder):
    if '.ipynb' not in filename:
        continue
    path = os.path.join(folder, filename)
    print(path)
    with open(path) as fh:
        notebook = json.load(fh)
    if 'colab' in notebook['metadata']:
        title_name = notebook['metadata']['colab']['name']
    print(f'The file {filename} calls itself {title_name}')
    for cell in notebook['cells']:
        if cell['cell_type'] == 'code':
            cell['execution_count'] = None
            cell['metadata']['cellView'] = 'form'
            cell['outputs'] = []
    with open(path, 'w') as fh:
        notebook = json.dump(notebook, fh, sort_keys=True, indent=4)

In [ ]:
names = [filename for filename in os.listdir(folder) if '.ipynb' in filename]
names

In [ ]:
# https://gist.github.com/matteoferla/7a2a8251a55cc34b9ae8d037b9e5bc80

from typing import *
from collections import defaultdict
from IPython.display import display, Markdown
import re


class ShieldBadgeMaker:
    """
    A single shield.io label / shield / flair generator built for customisation.
    There is a python package that does this already, but I wanted to copy paste the tags in a lazy way.
    ```python3
    badger = ShieldLBadgeMaker(user='matteoferla',
                          repo='Fragmenstein',
                          github_main='master')
    # arguments passed end up in self.data and can be accessed by subscripting the instance.
    # note that the python package name is assumed from repo:
    badger['packageName'] # fragmenstein
    # calling the instance will generate a markdown tag
    element = badger(template='/pypi/pyversions/:packageName', link=badger.PYPI)
    # some links are autogenerated, if a fake enum is passed, like the above.
    # this can be displayed in a Jupyter notebook thusly:
    display(Markdown(element))
    # show common repo stuff:
    badger.display_common_repo()
    # the arguments filled can be added after instantiation...
    badger = ShieldBadgerMaker()
    badger['so_id'] = 4625475
    badger['bio_se_id'] = 6322
    badger['orcid'] = '0000-0002-5508-4673'
    badger['twitter_handle'] = 'matteoferla'
    badger['scholar_id'] = 'gF-bp_cAAAAJ'
    lm.display_person()
    # or in a call:
    badger(template='/pypi/pyversions/:packageName', link=badger.PYPI, packageName='fragmenstein')
    # two special cases (as they use the "badges" void of any extras are:
    badger.get_doi(doi='10.1093/bioinformatics/btaa104')
    badger.get_colab(path='colab_fragmenstein.ipynb', label='Run In Colab')
    ```
    """
    # https://shields.io/

    # Faux enum
    GITHUB = 0
    LICENCE = 1
    PYPI = 2
    CODECLIMATE = 3
    STACKOVERFLOW = 4
    BIOINFO_STACKEXCHANGE = 5
    TWITTER = 6
    ORCID = 7
    DOI = 8
    SCHOLAR = 9
    COLAB = 10

    link_templates = {
        GITHUB:                'https://github.com/:user/:repo',
        LICENCE:               'https://github.com/:user/:repo/raw/{github_main}/LICENCE',
        PYPI:                  'https://pypi.org/project/:packageName',
        CODECLIMATE:           'https://codeclimate.com/github/:user/:repo',
        STACKOVERFLOW:         'https://stackoverflow.com/users/{so_id}',
        BIOINFO_STACKEXCHANGE: 'https://bioinformatics.stackexchange.com/users/{bioinfo_se_id}',
        TWITTER:               'https://twitter.com/{twitter_handle}',
        ORCID:                 'https://orcid.org/{orcid}',
        DOI:                   'https://doi.org/{doi}',
        SCHOLAR:               'https://scholar.google.com/citations?user={scholar_id}&hl=en',
    }
    icons = {
        # from https://github.com/simple-icons/simple-icons/blob/develop/slugs.md
        GITHUB:                'github',
        LICENCE:               'github',
        PYPI:                  'python',
        CODECLIMATE:           'codeclimate',
        STACKOVERFLOW:         'stackoverflow',
        BIOINFO_STACKEXCHANGE: 'stackexchange',
        TWITTER:               'twitter',
        ORCID:                 'orcid',
        SCHOLAR:               'googlescholar',
        COLAB:                 'googlecolab',
    }

    def __init__(self,
                 github_username: Optional[str] = None,
                 twitter_handle: str = '',
                 github_main: str = 'master',
                 python_package_name: Optional[str] = None,
                 orcid: str = '',
                 so_id: int = 0,
                 scholar_id: str = '',
                 bioinfo_se_id: int = 0,
                 **data):
        """
        Pass parameters as seen on https://shields.io
        Except some ambigous ones. `github_username` redirects to `user`
        while `twitter_handle` will require the template to have {twitter_handle}
        ditto for stackoverflow and other non GitHub users.
        """
        self.data = data
        self['github_main'] = github_main
        # user is ambiguous:
        if github_username:
            self['user'] = github_username
        self['twitter_handle'] = twitter_handle.replace('@', '')  # user is ambiguous!
        loc = locals()
        for k in ('orcid', 'so_id', 'bioinfo_se_id', 'scholar_id'):
            self[k] = loc[k]
        # hidden camelcase:
        if python_package_name:
            self['packageName'] = python_package_name
        elif 'packageName' not in self.data and 'repo' in self.data:
            self['packageName'] = self['repo'].lower()

    def __getitem__(self, key):
        if key in self.data:
            return self.data[key]
        raise ValueError(f'{key} not defined.')

    def __setitem__(self, key, value):
        self.data[key] = value

    def format(self, template: str, extras: dict):
        """
        Fills values by the normal {python} formatting and accepts the format from the API of shields.io
        eg. /github/release-date/:user/:repo
        """
        template = re.sub(r':(\w+)', r'{\1}', template)
        double_hyphanated = dict()
        for key, value in {**self.data, **extras}.items():
            if value is None:
                continue
            if isinstance(value, str) and '-' in value:
                value = value.replace('-', '--')
            if isinstance(value, str) and '/' in value:
                value = value.replace('/', '%2F')
            double_hyphanated[key] = value
        return template.format_map(double_hyphanated)

    def __call__(self,
                 template: str,
                 link: Union[str, int, None] = None,
                 a_label: Optional[str] = None,
                 **extras) -> str:
        if template[0] != '/':
            template = '/' + template
        if link not in self.icons:
            img_url = 'https://img.shields.io' + self.format(template, extras)
        elif '?' not in template:
            img_url = 'https://img.shields.io' + self.format(template, extras) + f'?logo={self.icons[link]}'
        else:
            img_url = 'https://img.shields.io' + self.format(template, extras) + f'&logo={self.icons[link]}'
        if not a_label:
            a_label = re.sub(r'\W+',' ', img_url)
        image_el = f'![{a_label}]({img_url})'
        if link is None:
            return image_el
        elif isinstance(link, str):
            link = self.format(link, extras)
            return f'[{image_el}]({link})'
        elif isinstance(link, int):
            if link in self.link_templates:
                link = self.format(self.link_templates[link], extras)
            else:
                raise ValueError(f'Invalid link value ({link})')
            return f'[{image_el}]({link})'
        else:
            TypeError(f'Invalid link type {type(link)}')

    def display_common_repo(self) -> List[str]:
        labels: List[str] = [
            self('pypi/v/:packageName', self.PYPI),
            self('/pypi/pyversions/:packageName', self.PYPI),
            self('/pypi/wheel/:packageName', self.PYPI),
            self('/pypi/format/:packageName', self.PYPI),
            self('/pypi/status/:packageName', self.PYPI),
            self('/pypi/dm/:packageName', self.PYPI),
            self('codeclimate/maintainability/:user/:repo', self.CODECLIMATE),
            self('codeclimate/issues/:user/:repo', self.CODECLIMATE),
            self('codeclimate/tech-debt/:user/:repo', self.CODECLIMATE),
            self('/github/forks/:user/:repo?label=Fork&style=social', self.GITHUB),
            self('/github/stars/:user/:repo?style=social', self.GITHUB),
            self('/github/watchers/:user/:repo?label=Watch&style=social', self.GITHUB),
            self('/github/last-commit/:user/:repo', self.GITHUB),
            self('github/license/:user/:repo', self.LICENCE),
            self('/github/release-date/:user/:repo', self.GITHUB),
            self('/github/commit-activity/m/:user/:repo', self.GITHUB),
            self('/github/issues/:user/:repo', self.GITHUB),
            self('/github/issues-closed/:user/:repo', self.GITHUB),
        ]
        for label in labels:
            print(label)
            display(Markdown(label))
        return labels

    def display_person(self) -> List[str]:
        labels: List[str] = [
            self.get_orcid(),
            self.get_scholar(),
            self('/twitter/follow/{twitter_handle}?label=Follow', self.TWITTER),
            self('/stackexchange/stackoverflow/r/{so_id}', self.STACKOVERFLOW),
            self('/stackexchange/bioinformatics/r/{bio_se_id}', self.BIOINFO_STACKEXCHANGE),
        ]
        for label in labels:
            print(label)
            display(Markdown(label))
        return labels
    
    def get_scholar(self, scholar_id: Optional[str]=None) -> str:
        if scholar_id:
            self['scholar_id'] = scholar_id
        return self('badge/google--scholar-{scholar_id}-success', link=self.SCHOLAR)
    
    def get_orcid(self, orcid: Optional[str]=None) -> str:
        if orcid:
            self['orcid'] = orcid
        return self('badge/orcid-{orcid}-a6ce39', link=self.ORCID)
    
    def get_doi(self, doi: str) -> str:
        """
        >>> badger('badge/doi-{doi}-fcb426', self.DOI, doi='10.1093/bioinformatics/btaa104')
        
        Disguisting yellow is from the doi website.
        """
        return self('badge/doi-{doi}-fcb426', link=self.DOI, doi=doi)
        
    def get_colab(self, path:str, label='Open In Colab', **extras) -> str:
        """
        >>> badger.get_colab(path='colab_fragmenstein.ipynb', label='Run')
        """
        img_el = f'![{label}](https://colab.research.google.com/assets/colab-badge.svg)'
        assert '.ipynb' in path, f'Path {path} is not a notebook'
        url =  self.format('https://colab.research.google.com/github/{user}/{repo}/blob/{github_main}/'+path, extras)
        return self(template='badge/colabs-{path}-f9ab00?logo='+self.icons[self.COLAB],
                    link=url,
                    path=path
                   )
    
    def get_mailhide_email(self, email_url:str, 
                           email_domain:str) -> str:
        """
        Do not set email_url to mailto:me@domain.com
        due to bots
        use https://mailhide.io/en is a successor in spirit to Google's mailhide.
        """        
        return self(f'badge/email-{email_domain}-informational&logo=googlemail', 
                    link=email_url,
                    email_domain=email_domain
                   )

In [ ]:
maker = ShieldBadgeMaker(user='matteoferla',
                      repo='pyrosetta_help',
                      github_main='master')
maker.display_common_repo()